# Manufacturing Causal-Net 

The target of the notebook is to learn *causal relations* in a dataset generated from a manufacturing simulator and, secondly, apply some *do-calculus operation* to observe effects of potential intervetions.

The simulator has been built using *Simpy*. Furhter information about Simpy in the following link: https://simpy.readthedocs.io/en/latest/index.html

The tools used are: 
- Pandas for data import and manipulation
- CausalNex for causal learning

In particular, "*CausalNex is a Python library that uses Bayesian Networks to combine machine learning and domain expertise for causal reasoning*". 

The relative documentation is available here: https://causalnex.readthedocs.io/en/latest/index.html

## Preparing the workspace
All the necessary tools has been downloaded and installed via the venv associated to the project. 
Refer to the requirements.txt file for further information.


### Importing necessary libraries
All the other necessary lirbaries are imported. They are: 
- os, time, datetime: for file handling operations 
- pandas: for data import, cleaning and input for CausalNex 
- causalnex.structure.notears: for network generation from pandas data
- networkx: for resulting network plotting

In [65]:
# Importing libraries
import shutil
import os
import time
import datetime
import networkx
import pandas
from causalnex.structure.notears import from_pandas

## Preparing the data set
The dataset generated from the simulation is exported as within a folder. 

The folder name is "yyyy.mm.dd-hh.mm-log", reporting the moment in time where the simulation started. The folder is placed inside of the Google Colab project folder.

The file containing the data is called "merged_logs.csv". It is imported as a Pandas Dataframe with the method "read_csv".

After the import, the head of the dataset and other dataset features are displayed.

---

The dataset used for the published experiment is "2022.03.14-11.54-log".

CORRECT THE LAST ERROR

In [ ]:
# Retrieving the last zip log file
# Directory list 
dir = os.listdir('..\manufacturing_model\logs')

el = 'none'
comp = 'none'
for i in range(len(dir)):
    if dir[i].endswith('.zip'):
        el = dir[i]
    
    if i > 0 and dir[i] > el and dir[i].endswith('.zip'): 
            el = dir[i]

# Selected folder
working_folder = el.replace('.zip', '')
zip_dataset_file = el
# Creating the folder to store the last zip file
os.mkdir('dataset\\' + working_folder)
# Moving the file
shutil.copyfile(src='..\manufacturing_model\logs\\' + zip_dataset_file, dst='dataset\\' + working_folder  + '\\' + zip_dataset_file)


In [67]:
# Setting the csv path 
CSV_PATH = working_folder 

CSV_PATH = 'dataset'

CSV_FILE_NAME = '/merged_logs.csv'
CSV_FILE_PATH = CSV_PATH + CSV_FILE_NAME

LIGHT_CSV_FILE_NAME = '/light-logs.csv'
LIGHT_CSV_FILE_PATH = CSV_PATH + LIGHT_CSV_FILE_NAME


In [68]:
# Preparing the data set
# Unzipping the folder
shutil.unpack_archive(zip_dataset_file, format='zip')

# Getting the dataframe from the file
data = pandas.read_csv(CSV_FILE_PATH, delimiter=',')

# Displaying the head and other dataset characteristics
print(data.head(10))
print('\n')
print(data.dtypes)
print('\n')
print(data.columns)


ReadError: 2022.09.12-17.51.zip is not a zip file

## Data cleaning and preparation
Since the dataset is virtually generated, NaN or missing value are not present.

Btw, some data preparation is computed. 

### Splitting the step column
The step column has the form of "step.moment": this is not really the meaning of the step used by Simpy. Is instead a "trick" to make logs and debugging easier in the previous phase.

So, for this phase, is more coherent to split this column into two different columns, namely "step" and "moment".

---

The split is performed in 3 steps: 
1. The "step" column is converted in type, from float to string. The result is saved in a new columns called "step_str" attached on the right to the initial dataframe. 
2. Using the str.split method, the "step_str" column is split into 2 columns at the "." (*point*). The resulting columns are saved into 2 columns called "step" and "moment". 
3. The no more necessary temporary column "step_str" is dropped.

Finally, the columns are reordered keeping the new "step" and "moment" columns on the left of the dataset and converting them into int type. 

In [ ]:
# Splitting the "step" column into "step" and "moment" in 3 steps:
# 1. Converting the step col into string type
data["step_str"] = data["step"].astype(str)
# 2. Using str.split to split the col at the "."
data[["step", "moment"]] = data.step_str.str.split(".", expand = True)
# 3. Dropping the temp col
data.drop(columns=["step_str"], inplace=True)

# Reordering the result
data = data[["step", "moment", "failure Machine A", "Machine A flag", 
      "failure Machine B",  "Machine B flag", "failure Machine C", 
      "Machine C flag"]]

# Converting everything in int 
data = data.astype(int)

print(data)


## Dropping unnecessary columns for learning 
In the previous step, during the columns reordering, not all the columns of the initial dataset have been used. For that reason, the unused columns have been dropped. 

So, there are only 2 columns left to be dropped: "step" and "moment". 

They are not necessary since the Bayesian Network used are not time-dependent: they just analyse the datast line-to-line trying to understand relations between features, without taking into account the time. 

In the following, "step" and "moment" are dropped.

In [ ]:
# Dropping the unneccessary columns
data.drop(columns=["step", "moment"], inplace=True)
print(data)


## Saving the cleaned dataset in a dedicated CSV file
The processed dataset is now ready in order to be processed by CausalNex. 

Before launching the learning phase, the dataset is exported as a CSV file in the same file location of the initial dataset CSV. 

The file is called "light-logs.csv", because "lighter" with respect to the initial dataset. 

In [ ]:
# Saving the light dataset into csv
data.to_csv(LIGHT_CSV_FILE_PATH)


## Data analysis 
Before the learning phase, data is analysed in order to understand the impact of the dataset structure on the causal-learning. 

The structure of the dataset, in fact, will let us understand in advance if some relationships will be caught or not. 

To do so, the following metrics will be computed: 
- fault_Machine x = 1 AND flag_Machine_x = 1 AND flag_Machine_C = 1
- fault_Machine x = 1 AND flag_Machine_x = 1 AND flag_Machine_C = 0

The ratio between those 2 metrics will let the user understand if the breakdown of an upstream machine (Machine A and B in the sim) has affected the downstream machine (Machine C in the sim). 

If the impact is high, the relation is likely to be caught. 


In [ ]:
# Loading the light dataset into csv
data = pandas.read_csv(LIGHT_CSV_FILE_PATH, delimiter=',', index_col=0)
data


In [ ]:
# Machine A Ratio
fault_A_flag = data[(data['failure Machine A'] == 1) & 
                    (data['Machine A flag'] == 1)].count()

fault_A_flag_C_High = data[(data['failure Machine A'] == 1) & 
                           (data['Machine A flag'] == 1) & 
                           (data['Machine C flag'] == 1)].count()

fault_A_flag_C_Low = data[(data['failure Machine A'] == 1) & 
                          (data['Machine A flag'] == 1) & 
                          (data['Machine C flag'] == 0)].count()

ratio_A_high = fault_A_flag_C_High[0]/fault_A_flag[0]
ratio_A_low = fault_A_flag_C_Low[0]/fault_A_flag[0]

print('Ratio A High: ', ratio_A_high)
print('Ratio A Low: ', ratio_A_low)


In [ ]:
# Machine B Ratio
fault_B_flag = data[(data['failure Machine B'] == 1) & 
                    (data['Machine B flag'] == 1)].count()

fault_B_flag_C_High = data[(data['failure Machine B'] == 1) & 
                           (data['Machine B flag'] == 1) & 
                           (data['Machine C flag'] == 1)].count()

fault_B_flag_C_Low = data[(data['failure Machine B'] == 1) & 
                          (data['Machine B flag'] == 1) & 
                          (data['Machine C flag'] == 0)].count()

ratio_B_high = fault_B_flag_C_High[0]/fault_B_flag[0]
ratio_B_low = fault_B_flag_C_Low[0]/fault_B_flag[0]

print('Ratio B High: ', ratio_B_high)
print('Ratio B Low: ', ratio_B_low)


## CausalNex application: applying the causal-network


In [ ]:
# Eliminating spaces from column names - useful for later
column_dict = {}
for el in data.columns:
  key = el
  el = el.replace(' ', '_')
  column_dict [key] = el

data = data.rename(columns=column_dict)

# Getting tabu child nodes
tabu_child_list = [x for x in data.columns if 'failure' in x]

print('Tabu child list: ', tabu_child_list)

# Training the model
# Declaring and mining the structure of the causal-net
start_time = time.time()
structure_model = from_pandas(data, tabu_child_nodes=tabu_child_list)
finish_time = time.time()
sim_time = finish_time - start_time

# Printing the structure model with Python
print("Total training time: {} min, {} secs".format(round(sim_time/60, 0), 
                                                    round(sim_time%60, 0)))
# print("Total training time: {} seconds".format(round(sim_time, 2)/60))
print(structure_model)


In [ ]:
data.shape

In [ ]:
# Printing the structure model with networkx
networkx.draw(structure_model, with_labels=True)


In [ ]:
structure_model.remove_edges_below_threshold(0.1)
print(structure_model)
# Printing the structure model again
networkx.draw(structure_model, with_labels=True)


In [ ]:
structure_model.remove_edges_below_threshold(0.2)
print(structure_model)
# Printing the structure model again
networkx.draw(structure_model, with_labels=True)


In [ ]:
structure_model.remove_edges_below_threshold(0.3)
print(structure_model)
# Printing the structure model again
networkx.draw(structure_model, with_labels=True)


In [ ]:
# Exporting the model at the best threshold (as said in the NOTEARS arxiv paper)
networkx.drawing.nx_pydot.write_dot(structure_model, CSV_PATH + '/graph.dot')

In [ ]:
# Getting weights of the generated model in Python dict structure
edges_weights_dict = networkx.to_dict_of_dicts(structure_model)

# Saving the weights of the generated model
with open(CSV_PATH + '/edges-weights.txt', 'a') as f:
  for k1 in edges_weights_dict:
    for k2 in edges_weights_dict[k1]:
      text = k1 + ' -> ' + k2 + ': ' \
            + str(edges_weights_dict[k1][k2].get('weight'))
    f.write(text)
  f.close()


In [ ]:
# Bayesian network instantiation
from causalnex.network import BayesianNetwork

bayesian_net = BayesianNetwork(structure_model)


In [ ]:
from sklearn.model_selection import train_test_split

# Split 90% train and 10% test
training_data = data.copy()
train, test = train_test_split(training_data, train_size=0.9, test_size=0.1, 
                               random_state=7)

# Fitting node states into the Bayesian Network: here they are inferred from the 
# input data, but sometimes is necessary to provide a dictionary for them to be 
# assigned
bayesian_net = bayesian_net.fit_node_states(training_data)

# Fitting the data into the prepared net
bayesian_net = bayesian_net.fit_cpds(train, method="BayesianEstimator", 
                                     bayes_prior="K2")

bayesian_net.cpds


In [ ]:
test


In [ ]:
ser = test.groupby(["failure_Machine_A"]).size()
ser 


In [ ]:
from causalnex.evaluation import classification_report, roc_auc

# Model quality evaluation for failure Machine A
# classification_report(bayesian_net, test, "failure_Machine_A")

# ROC, AUC curve computation for failure Machine A
machine_A_failure_roc, machine_A_failure_auc = roc_auc(bayesian_net, test, "failure_Machine_A")
#print('Machine A failure Node ROC: ', machine_A_failure_roc)
print('Machine A failure Node AUC: ', machine_A_failure_auc)

# Model quality evaluation for Machine A flag
# classification_report(bayesian_net, test, "Machine_A_flag")

# ROC, AUC curve computation for Machine A flag
machine_A_flag_roc, machine_A_flag_auc = roc_auc(bayesian_net, test, "Machine_A_flag")
#print('Machine A flag Node ROC: ', machine_A_flag_roc)
print('Machine A flag Node AUC: ', machine_A_flag_auc)

# Model quality evaluation for failure Machine B
# classification_report(bayesian_net, test, "failure_Machine_B")

# ROC, AUC curve computation for failure Machine B
machine_B_failure_roc, machine_B_failure_auc = roc_auc(bayesian_net, test, "failure_Machine_B")
#print('Machine B failure Node ROC: ', machine_B_failure_roc)
print('Machine B failure Node AUC: ', machine_B_failure_auc)

# Model quality evaluation for Machine B flag
# classification_report(bayesian_net, test, "Machine_B_flag")

# ROC, AUC curve computation for Machine B flag
machine_B_flag_roc, machine_B_flag_auc = roc_auc(bayesian_net, test, "Machine_B_flag")
#print('Machine B flag Node ROC: ', machine_B_flag_roc)
print('Machine B flag Node AUC: ', machine_B_flag_auc)

# Model quality evaluation for failure Machine C
# classification_report(bayesian_net, test, "failure_Machine_C")

# ROC, AUC curve computation for failure Machine C
machine_C_failure_roc, machine_C_failure_auc = roc_auc(bayesian_net, test, "failure_Machine_C")
#print('Machine C failure node Node ROC: ', machine_C_failure_roc)
print('Machine C failure node Node AUC: ', machine_C_failure_auc)

# Model quality evaluation for Machine C flag
# classification_report(bayesian_net, test, "Machine_C_flag")

# ROC, AUC curve computation for Machine C flag
machine_C_flag_roc, machine_C_flag_auc = roc_auc(bayesian_net, test, "Machine_C_flag")
#print('Machine C flag Node ROC: ', machine_C_flag_roc)
print('Machine C flag Node AUC: ', machine_C_flag_auc)


In [ ]:
# Model quality evaluation for failure Machine A
classification_report(bayesian_net, test, "failure_Machine_A")


In [ ]:
# Model quality evaluation for Machine A flag
classification_report(bayesian_net, test, "Machine_A_flag")


In [ ]:
# Model quality evaluation for failure Machine B
classification_report(bayesian_net, test, "failure_Machine_B")


In [ ]:
# Model quality evaluation for Machine B flag
classification_report(bayesian_net, test, "Machine_B_flag")


In [ ]:
# Model quality evaluation for failure Machine C
classification_report(bayesian_net, test, "failure_Machine_C")


In [ ]:
# Model quality evaluation for Machine C flag
classification_report(bayesian_net, test, "Machine_C_flag")


In [ ]:
# Fitting with the whole dataset
bayesian_net = bayesian_net.fit_cpds(data, method="BayesianEstimator", bayes_prior="K2")


In [ ]:
from causalnex.inference import InferenceEngine

ie = InferenceEngine(bayesian_net)
marginals = ie.query()
marginals["Machine_C_flag"]


In [ ]:
# Calculating conditioned probability Machine A
p_break_given_flag_A = ie.query({"Machine_A_flag": 1})
print("Marginal Failure Machine A | Machine A flag", p_break_given_flag_A["failure_Machine_A"])

# Calculating conditioned probability Machine B
p_break_given_flag_B = ie.query({"Machine_B_flag": 1})
print("Marginal Failure Machine B | Machine B flag", p_break_given_flag_B["failure_Machine_B"])

# Calculating conditioned probability Machine C
p_break_given_flag_C = ie.query({"Machine_C_flag": 1})
print("Marginal Failure Machine C | Machine C flag", p_break_given_flag_C["failure_Machine_C"])


In [ ]:
# Calculating conditioned probability Machine C given failure A
p_break_A_given_flag_C = ie.query({"Machine_C_flag": 1})
print("Marginal Failure Machine A | Machine C flag", p_break_A_given_flag_C["failure_Machine_A"])

# Calculating conditioned probability Machine C given failure B
p_break_B_given_flag_C = ie.query({"Machine_C_flag": 1})
print("Marginal Failure Machine B | Machine C flag", p_break_B_given_flag_C["failure_Machine_B"])


In [ ]:
# Calculating conditioned probability Machine C given failure A
p_flag_A_given_flag_C = ie.query({"Machine_C_flag": 1})
print("Marginal A flag | Machine C flag", p_break_A_given_flag_C["Machine_A_flag"])

# Calculating conditioned probability Machine C given failure B
p_flag_B_given_flag_C = ie.query({"Machine_C_flag": 1})
print("Marginal B flag | Machine C flag", p_break_B_given_flag_C["Machine_B_flag"])


In [ ]:
# Do calculus - Failure A vs Flag A
print('distribution before do failure A', ie.query()['Machine_A_flag'])
ie.do_intervention('failure_Machine_A', {1: 1.0, 0: 0.0})
print('distribution after do failure A', ie.query()['Machine_A_flag'])
ie.reset_do('failure_Machine_A')
print('\n')

# Do calculus - Failure B vs Flag B
print('distribution before do failure B', ie.query()['Machine_B_flag'])
ie.do_intervention('failure_Machine_B', {1: 1.0, 0: 0.0})
print('distribution after do failure B', ie.query()['Machine_B_flag'])
ie.reset_do('failure_Machine_B')
print('\n')

# Do calculus - Failure C vs Flag C
print('distribution before do failure C', ie.query()['Machine_C_flag'])
ie.do_intervention('failure_Machine_C', {1: 1.0, 0: 0.0})
print('distribution after do failure C', ie.query()['Machine_C_flag'])
ie.reset_do('failure_Machine_C')
print('\n')


In [ ]:
# Do calculus - Failure A vs Flag C
print('distribution before do failure A on flag C', ie.query()['Machine_C_flag'])
ie.do_intervention('failure_Machine_A', {1: 1.0, 0: 0.0})
print('distribution after do failure A on flag C', ie.query()['Machine_C_flag'])
ie.reset_do('failure_Machine_A')
print('\n')

# Do calculus - Failure B vs Flag C
print('distribution before do failure B on flag C', ie.query()['Machine_C_flag'])
ie.do_intervention('failure_Machine_B', {1: 1.0, 0: 0.0})
print('distribution after do failure B on flag C', ie.query()['Machine_C_flag'])
ie.reset_do('failure_Machine_B')
print('\n')


In [ ]:
# Do calculus - Flag A vs Flag C
print('distribution before do flag A on flag C', ie.query()['Machine_C_flag'])
ie.do_intervention('Machine_A_flag', {1: 1.0, 0: 0.0})
print('distribution after do flag A on flag C', ie.query()['Machine_C_flag'])
ie.reset_do('Machine_A_flag')
print('\n')

# Do calculus - Flag B vs Flag C
print('distribution before do flag B on flag C', ie.query()['Machine_C_flag'])
ie.do_intervention('Machine_B_flag', {1: 1.0, 0: 0.0})
print('distribution after do flag B on flag C', ie.query()['Machine_C_flag'])
ie.reset_do('Machine_B_flag')
print('\n')